In [772]:
# 주요 라이브러리 import
import warnings
warnings.filterwarnings(action='ignore')
import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import Ridge, Lasso, ElasticNet

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:/Windows/Fonts/malgun.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [773]:
df=pd.read_csv('2018 국민여행조사_국내여행.csv')

In [774]:
df['D_TRA1_SYEAR'].value_counts()

2018.0    14296
2017.0        7
Name: D_TRA1_SYEAR, dtype: int64

In [775]:
print(df['D_TRA1_SDAY'].isna().sum(), df['D_TRA1_COST'].isna().sum(), 
df['A1'].isnull().sum(), df['A1'].notnull().sum(), df['D_TRA1_CASE'].notnull().sum())

9851 9851 12306 11848 14303


In [776]:
# 여행 유형 미체크 항목을 0으로 채움
df['D_TRA1_CASE'].fillna(0, inplace=True)
df['D_TRA2_CASE'].fillna(0, inplace=True)
df['D_TRA3_CASE'].fillna(0, inplace=True)
df['D_TRA4_CASE'].fillna(0, inplace=True)

In [777]:
df['D_TRA1_CASE'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
24149    False
24150    False
24151    False
24152    False
24153    False
Name: D_TRA1_CASE, Length: 24154, dtype: bool

In [778]:
# 여행 유형이 "2.국내여행"인 레코드만 추출
df=df[(df['D_TRA1_CASE']==2) | (df['D_TRA1_CASE']==0)]
df=df[(df['D_TRA2_CASE']==2) | (df['D_TRA2_CASE']==0)]
df=df[(df['D_TRA3_CASE']==2) | (df['D_TRA3_CASE']==0)]
df=df[(df['D_TRA4_CASE']==2) | (df['D_TRA4_CASE']==0)]

In [779]:
df.groupby('D_TRA1_CASE').count()

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_SYEAR,D_TRA1_SMONTH,D_TRA1_SDAY,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
D_TRA1_CASE,,,,,,,,,,,,,,,,,,,,,
0.0,9778,9778,9778,9778,756,756,756,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,10813,10813,10813,10813,936,936,936,10813,10813,10813,...,0,0,0,0,0,0,0,0,0,0


In [780]:
# 여행 유형이 "2.국내여행"인 레코드만 추출
# df=df[df['D_TRA1_CASE']==2 | df['D_TRA2_CASE']==2]
# df=df[(df['D_TRA2_CASE']==2)]
# df=df[df['D_TRA3_CASE']==2]
# df=df[df['D_TRA4_CASE']==2]

In [781]:
## 추천도, 재방문의사 보통이상만 추출
df = df[(df['A12']>=3) & (df['A13']>=3)]

In [782]:
## 1차 필요컬럼
columns_list = [
#   ============================ 재방문 의도, 타인추천 의도
    'A13',
    'A14',
#   ============================ 여행지에서의 활동
    'A3_1',
    'A3_2',
    'A3_3',
    'A3_4',
    'A3_5',
    'A3_6',
    'A3_7',
    'A3_8',
    'A3_9',
    'A3_10',
    'A3_11',
    'A3_12',
    'A3_12',
    'A3_13',
    'A3_14',
    'A3_15',
    'A3_16',
    'A3_17',
    'A3_18',
    'A3_19',
    'A3_20',
    'A3_21',
#   ============================ 여행 시작 날짜, 종료 날짜, 방문 지역, 각 방문 지역의 시작, 종료 날짜
    'D_TRA1_SYEAR', 
    'D_TRA1_SMONTH',
    'D_TRA1_SDAY',
    'D_TRA1_EYEAR',
    'D_TRA1_EMONTH',
    'D_TRA1_EDAY',
    
    'D_TRA1_1_CITY1',
    'D_TRA1_1_TOWN1',
    'D_TRA1_1_CITY2',
    'D_TRA1_1_TOWN2',
    'D_TRA1_1_CITY3',
    'D_TRA1_1_TOWN3',
    'D_TRA1_1_SYEAR',
    'D_TRA1_1_SMONTH',
    'D_TRA1_1_SDAY',
    'D_TRA1_1_EYEAR',
    'D_TRA1_1_EMONTH',
    'D_TRA1_1_EDAY',
    'D_TRA1_2_CITY1',
    'D_TRA1_2_TOWN1',
    'D_TRA1_2_SYEAR',
    'D_TRA1_2_SMONTH',
    'D_TRA1_2_SDAY',
    'D_TRA1_2_EYEAR',
    'D_TRA1_2_EMONTH',
    'D_TRA1_2_EDAY',
#   ============================  
    'D_TRA2_SYEAR', 
    'D_TRA2_SMONTH',
    'D_TRA2_SDAY',
    'D_TRA2_EYEAR',
    'D_TRA2_EMONTH',
    'D_TRA2_EDAY',
    
    'D_TRA2_1_CITY1',
    'D_TRA2_1_TOWN1',
    'D_TRA2_1_CITY2',
    'D_TRA2_1_TOWN2',
    'D_TRA2_1_CITY3',
    'D_TRA2_1_TOWN3',
    'D_TRA2_1_SYEAR',
    'D_TRA2_1_SMONTH',
    'D_TRA2_1_SDAY',
    'D_TRA2_1_EYEAR',
    'D_TRA2_1_EMONTH',
    'D_TRA2_1_EDAY',
    'D_TRA2_2_CITY1',
    'D_TRA2_2_TOWN1',
    'D_TRA2_2_SYEAR',
    'D_TRA2_2_SMONTH',
    'D_TRA2_2_SDAY',
    'D_TRA2_2_EYEAR',
    'D_TRA2_2_EMONTH',
    'D_TRA2_2_EDAY',
#   ============================    
    'D_TRA3_SYEAR', 
    'D_TRA3_SMONTH',
    'D_TRA3_SDAY',
    'D_TRA3_EYEAR',
    'D_TRA3_EMONTH',
    'D_TRA3_EDAY',
    
    'D_TRA3_1_CITY1',
    'D_TRA3_1_TOWN1',
    'D_TRA3_1_CITY2',
    'D_TRA3_1_TOWN2',
    'D_TRA3_1_CITY3',
    'D_TRA3_1_TOWN3',
    'D_TRA3_1_SYEAR',
    'D_TRA3_1_SMONTH',
    'D_TRA3_1_SDAY',
    'D_TRA3_1_EYEAR',
    'D_TRA3_1_EMONTH',
    'D_TRA3_1_EDAY',
    'D_TRA3_2_CITY1',
    'D_TRA3_2_TOWN1',
    'D_TRA3_2_SYEAR',
    'D_TRA3_2_SMONTH',
    'D_TRA3_2_SDAY',
    'D_TRA3_2_EYEAR',
    'D_TRA3_2_EMONTH',
    'D_TRA3_2_EDAY',
#   ============================    
    'D_TRA4_SYEAR', 
    'D_TRA4_SMONTH',
    'D_TRA4_SDAY',
    'D_TRA4_EYEAR',
    'D_TRA4_EMONTH',
    'D_TRA4_EDAY',
    
    'D_TRA4_1_CITY1',
    'D_TRA4_1_TOWN1',
    'D_TRA4_1_CITY2',
    'D_TRA4_1_TOWN2',
    'D_TRA4_1_CITY3',
    'D_TRA4_1_TOWN3',
    'D_TRA4_1_SYEAR',
    'D_TRA4_1_SMONTH',
    'D_TRA4_1_SDAY',
    'D_TRA4_1_EYEAR',
    'D_TRA4_1_EMONTH',
    'D_TRA4_1_EDAY',
    'D_TRA4_2_CITY1',
    'D_TRA4_2_TOWN1',
    'D_TRA4_2_SYEAR',
    'D_TRA4_2_SMONTH',
    'D_TRA4_2_SDAY',
    'D_TRA4_2_EYEAR',
    'D_TRA4_2_EMONTH',
    'D_TRA4_2_EDAY',
#   ============================  사전예약사항(숙소 등)
    'A8_1',
    'A8_2',
    'A8_3',
    'A8_4',
    'A8_5',
    'A8_6',
    'A8_7',
    'A8_10',
    'A8_11',
#   ============================  주요 이동 교통수단 1~3순위
    'A5_1',
    'A5_2',
    'A5_3',
#   ============================  여행지 선택 이유 1~3순위
    'A2_1',
    'A2_2',
    'A2_3',
#   ============================  동반자 유형
    'A7B_1',
    'A7B_2',
    'A7B_3',
    'A7B_4',
    'A7B_5',
    'A7B_6',
    'A7B_7',
#   ============================  여행 인원수
    'A7',
#   ============================  15세 미만 동만자 수
    'A7A',
#   ============================  여행지 지출 총합
    'N_A10',
#   ============================  전반적 만족도
    'A12',
#   ============================  성별
    'SEX',
#   ============================  나이
    'AGE_1YEAR_UNIT',
#   ============================  출발 월
    'BMON',
#   ============================  거주지
    'BSIDO'
]

In [783]:
# 컬럼명 맞게 들어갔는지 체크
for one in columns_list:
    df[one].head(1)

In [784]:
df.columns

Index(['ID', 'MON_EXP_DOM_TOUR', 'MON_EXP_DOM_BUSIN', 'MON_EXP_DOM_FRI',
       'YEAR_EXP_DOM_TOUR', 'YEAR_EXP_DOM_BUSIN', 'YEAR_EXP_DOM_FRI',
       'D_TRA1_CASE', 'D_TRA1_SYEAR', 'D_TRA1_SMONTH',
       ...
       '국내_여행방문지_기타당일7', '국내_여행방문지_기타당일8', '국내_여행방문지_기타당일9', '국내_여행방문지_기타당일10',
       '국내_여행방문지_기타당일11', '국내_여행방문지_기타당일12', '국내_여행방문지_기타당일13',
       '국내_여행방문지_기타당일14', '국내_여행방문지_기타당일15', '국내_여행방문지_기타당일16'],
      dtype='object', length=1302)

In [785]:
df[columns_list]

,A13,A14,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,...,A7B_6,A7B_7,A7,A7A,N_A10,A12,SEX,AGE_1YEAR_UNIT,BMON,BSIDO
3,5.0,4.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,360000.000000,4.0,2,62,7,11
5,4.0,3.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,33.0,0.0,57200.000000,4.0,2,53,5,11
6,4.0,4.0,1.0,2.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,5.0,0.0,90000.000000,4.0,1,76,5,11
9,4.0,3.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,68500.000000,4.0,2,66,5,11
10,4.0,4.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,42003.514938,4.0,2,65,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24114,3.0,3.0,1.0,NaN,3.0,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,3.0,1.0,45283.018868,4.0,2,46,8,39
24115,4.0,2.0,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,1.0,NaN,318181.818182,4.0,2,54,8,39
24126,4.0,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,5.0,0.0,30000.000000,4.0,2,16,12,39
24132,4.0,4.0,1.0,2.0,NaN,NaN,5.0,6.0,NaN,NaN,...,NaN,NaN,4.0,2.0,127941.176471,4.0,2,26,9,39


In [786]:
df[df["D_TRA1_1_SYEAR"]!=df['D_TRA1_1_EYEAR']].head()

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
1165,11130700151_1223104,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,11130762371_1201154,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,1114070066A_101092,1,2,2,NaN,NaN,NaN,2.0,2017.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,11240530281_1202386,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2877,21010540601_1202813,1,2,2,1.0,2.0,2.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [787]:
df[df["D_TRA1_SMONTH"]!=df['D_TRA1_EMONTH']]

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
1165,11130700151_1223104,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1176,1113071084A_703602,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,11130762371_1201154,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,1114070066A_101092,1,2,2,NaN,NaN,NaN,2.0,2017.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,11240530281_1202386,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23463,39010120311_724855,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23488,39010121281_1222623,1,2,2,1.0,2.0,2.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23491,39010121361_747628,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23794,3901060089A_748008,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [788]:
# 여행 박 수 계산을 위한 라이브러리 import
from datetime import datetime

In [789]:
now  = datetime.now()
date_to_compare = datetime.strptime("2022512", "%Y%m%d")
date_diff = now - date_to_compare
print("차이 :", date_diff.days)
type(date_diff.days)

차이 : 4


int

In [790]:
df.loc[6,'D_TRA1_SYEAR']

2018.0

In [791]:
# df['D_TRA1_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA2_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA3_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA4_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA1_EYEAR'].fillna(2018, inplace=True)
# df['D_TRA2_EYEAR'].fillna(2018, inplace=True)
# df['D_TRA3_EYEAR'].fillna(2018, inplace=True)
# df['D_TRA4_EYEAR'].fillna(2018, inplace=True)

In [792]:
df['D_TRA1_1_SYEAR'].isna().sum()

23

In [793]:
df['D_TRA2_1_SYEAR'].isna().sum()

10584

In [794]:
df['D_TRA2_1_SMONTH'].isna().sum()

10584

In [795]:
df['D_TRA2_1_SDAY'].isna().sum()

10584

## 1~4번 여행지 시작일, 종료일 컬럼 각각 추가 (임시 데이터프레임 생성)

In [796]:
date_df=pd.DataFrame()

In [797]:
# 날짜 자료형 object -> int64로 변경

In [798]:
df['D_TRA1_SYEAR'].fillna(-1).astype('int64').replace({-1: np.nan}).astype(str)

3        2018.0
5        2018.0
6        2018.0
9        2018.0
10       2018.0
          ...  
24114    2018.0
24115    2018.0
24126    2018.0
24132    2018.0
24137    2018.0
Name: D_TRA1_SYEAR, Length: 10765, dtype: object

In [799]:
s='안녕하세요'

In [800]:
s[:-2]

'안녕하'

#### 소수점 버리기 함수

In [801]:
def cut_zero(str):
    return str[:-2]

In [802]:
df['D_TRA1_SYEAR'].astype(str).apply(cut_zero)

3        2018
5        2018
6        2018
9        2018
10       2018
         ... 
24114    2018
24115    2018
24126    2018
24132    2018
24137    2018
Name: D_TRA1_SYEAR, Length: 10765, dtype: object

#### 한자리 숫자 앞에 0붙이기 함수

In [829]:
def add_zero(str):
    if len(str)>1:
        return str
    else:
        return '0'+str

In [830]:
df['D_TRA1_SMONTH'].astype(str).apply(cut_zero)

3         7
5         5
6         5
9         5
10        9
         ..
24114     8
24115     8
24126    12
24132     9
24137     6
Name: D_TRA1_SMONTH, Length: 10765, dtype: object

In [831]:
df['D_TRA1_SDAY'].astype(str).apply(cut_zero).apply(add_zero)

3        06
5        19
6        08
9        04
10       22
         ..
24114    03
24115    21
24126    25
24132    03
24137    07
Name: D_TRA1_SDAY, Length: 10765, dtype: object

In [832]:
df['D_TRA2_SYEAR']

3       NaN
5       NaN
6       NaN
9       NaN
10      NaN
         ..
24114   NaN
24115   NaN
24126   NaN
24132   NaN
24137   NaN
Name: D_TRA2_SYEAR, Length: 10765, dtype: float64

In [835]:
for i in range(1, 5):
    date_df['start'+str(i)]=df['D_TRA'+str(i)+'_SYEAR'].astype(str).apply(cut_zero).apply(add_zero)+df['D_TRA'+str(i)+'_SMONTH'].astype(str).apply(cut_zero).apply(add_zero)+df['D_TRA'+str(i)+'_SDAY'].astype(str).apply(cut_zero).apply(add_zero)

In [836]:
date_df.head(2)

,start1,start2,start3,start4,end1,end2,end3,end4,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day
3,20180706,0n0n0n,0n0n0n,0n0n0n,201878,nnn,nnn,nnn,2,None,None,None
5,20180519,0n0n0n,0n0n0n,0n0n0n,2018519,nnn,nnn,nnn,0,None,None,None


In [837]:
for i in range(1, 5):
    date_df['end'+str(i)]=df['D_TRA'+str(i)+'_EYEAR'].astype(str).apply(cut_zero).apply(add_zero)+df['D_TRA'+str(i)+'_EMONTH'].astype(str).apply(cut_zero).apply(add_zero)+df['D_TRA'+str(i)+'_EDAY'].astype(str).apply(cut_zero).apply(add_zero)

In [838]:
date_df.head(2)

,start1,start2,start3,start4,end1,end2,end3,end4,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day
3,20180706,0n0n0n,0n0n0n,0n0n0n,20180708,0n0n0n,0n0n0n,0n0n0n,2,None,None,None
5,20180519,0n0n0n,0n0n0n,0n0n0n,20180519,0n0n0n,0n0n0n,0n0n0n,0,None,None,None


In [839]:
date_df['start2'].value_counts()

0n0n0n      10584
20181020       10
20181124        9
20181021        8
20181028        6
            ...  
20180512        1
20180130        1
20180316        1
20180215        1
20181103        1
Name: start2, Length: 101, dtype: int64

#### datetime으로 형변환 함수

In [840]:
def toDate(str):
    return datetime.strptime(str, "%Y%m%d")

## datetime으로 날짜 차이 계산하여 박수 컬럼 추가

In [841]:
for i in range(1, 5):
    date_df['D_TRA'+str(i)+'_S_Day']=None

for i in range(0, 24138):
    try:
        differ=toDate(date_df['end1'][i])-toDate(date_df['start1'][i])
        date_df['D_TRA1_S_Day'][i]=differ.days
    except:
        pass
    
    try:
        differ=toDate(date_df['end2'][i])-toDate(date_df['start2'][i])
        date_df['D_TRA2_S_Day'][i]=differ.days
    except:
        pass
    
    try:
        differ=toDate(date_df['end3'][i])-toDate(date_df['start3'][i])
        date_df['D_TRA3_S_Day'][i]=differ.days
    except:
        pass
    
    try:
        differ=toDate(date_df['end4'][i])-toDate(date_df['start4'][i])
        date_df['D_TRA4_S_Day'][i]=differ.days
    except:
        pass

In [842]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10765 entries, 3 to 24137
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   start1        10765 non-null  object
 1   start2        10765 non-null  object
 2   start3        10765 non-null  object
 3   start4        10765 non-null  object
 4   end1          10765 non-null  object
 5   end2          10765 non-null  object
 6   end3          10765 non-null  object
 7   end4          10765 non-null  object
 8   D_TRA1_S_Day  10742 non-null  object
 9   D_TRA2_S_Day  181 non-null    object
 10  D_TRA3_S_Day  17 non-null     object
 11  D_TRA4_S_Day  3 non-null      object
dtypes: object(12)
memory usage: 1.3+ MB


In [843]:
date_df[date_df['D_TRA1_S_Day']>20]

,start1,start2,start3,start4,end1,end2,end3,end4,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day
10232,20180221,0n0n0n,0n0n0n,0n0n0n,20180317,0n0n0n,0n0n0n,0n0n0n,24,None,None,None


In [844]:
df[['D_TRA1_S_Day','D_TRA2_S_Day','D_TRA3_S_Day','D_TRA4_S_Day']]=date_df[['D_TRA1_S_Day','D_TRA2_S_Day','D_TRA3_S_Day','D_TRA4_S_Day']]

In [845]:
df[['D_TRA1_S_Day','D_TRA2_S_Day','D_TRA3_S_Day','D_TRA4_S_Day']]

,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day
3,2,None,None,None
5,0,None,None,None
6,0,None,None,None
9,2,None,None,None
10,1,None,None,None
...,...,...,...,...
24114,1,None,None,None
24115,1,None,None,None
24126,0,None,None,None
24132,2,None,None,None


## ===========================================

In [846]:
df[['D_TRA1_S_Day']][df['D_TRA1_S_Day']>300]

,D_TRA1_S_Day


In [847]:
## 퍼진 컬럼들 압축 과정

## TRA1_1, TRA1_2 등 모든 여행지 파생변수 area로 ,로 구분해서 입력, 
## 여행지 개수 count하여 no_of_place 컬럼에입력
# 1-1 1-2 1-3 1-4 1-5 ... 8-10
# [1,2,3,4,5,6]
sleepnights_list = [] #여행지에서 잔 밤수(박) ex) 1박 2일 = 1
cost_list = [] # 여행비용 <=이후 중복컬럼 생겨서 미사용
area_list = [] # 거주지 
count_list = [] # 여행지 개수 일치확인용 
month_list = [] # 여행 출발 월
for index in df.index:
    sleepnights = ''
    cost = ''
    area = ''
    count = 0
    month = ''
    # accommodation = ''
    for i in range (1,5):
        col_sleepnights = 'D_TRA'+str(i)+'_S_Day'       
        col_cost = 'D_TRA'+str(i)+'_COST'        
        try: 
            sleepnights = sleepnights + str(int(df.loc[index,col_sleepnights])) + ','
            cost = cost + str(int(df.loc[index,col_cost])) +','
        except: pass
        col_month = 'D_TRA'+str(i)+'_SMONTH'
        try:
                month = month + str(int(df.loc[index, col_month])) + ','
        except:
                pass
        for j in range(1,3):

            col_area ='D_TRA'+str(i)+'_'+str(j)+'_TOWN1'
            
            
            try: 
                area = area + str(int(df.loc[index, col_area])) + ','
                count += 1
            except: pass
            
            # for k in range(1,15):
            #     col_accommodation = 'D_TRA'+str(i)+'_'+str(j)+'A6_'+str(k)
            #     try:
            #         accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
            #     except:
            #         pass
    try:
        area = area.rstrip(area[-1])
    except:pass
    try:
        sleepnights = sleepnights.rstrip(sleepnights[-1])
    except:pass
    try:
        cost = cost.rstrip(cost[-1])
    except:pass
    try:
        month = month.rstrip(month[-1])
    except:pass
    # try:
    #     accommodation = accommodation.rstrip(accommodation[-1])
    # except:pass
    area_list.append(area)
    count_list.append(count)
    month_list.append(month)
    cost_list.append(cost)
    # accommodation_list.append(accommodation)
    sleepnights_list.append(sleepnights)

df['area']= area_list
df['no_of_place'] = count_list
df['month'] = month_list
df['cost'] = cost_list
df['sleepnights'] = sleepnights_list
# df['accommodation'] = accommodation_list

In [848]:
df[['area','no_of_place','month','cost','sleepnights']][df['no_of_place']>1].head(50)

,area,no_of_place,month,cost,sleepnights
195,"39020,25040,25040",3,"5,5,5","500000,150000,397500","3,1,1"
978,"32310,32310",2,"6,6","165000,80000","1,1"
990,"31350,33370",2,"3,3","90000,51000","3,1"
1169,"21090,31240",2,"12,12","320000,70000","2,0"
1176,"32060,31120",2,7,200000,3
1213,"32060,31120",2,8,130000,2
1466,"32390,31370",2,7,150000,2
1573,"38110,37360",2,4,90000,0
1619,"31370,33380",2,"3,3","81000,100000","0,0"
2680,"32410,32410,32410",3,"12,12,12","150000,60000,150000","0,0,0"


In [849]:
accommodation_list = []
for index in df.index:
    accommodation = ''
    for k in range(1,12):
        col_accommodation = 'A8_'+str(k)
        try:
            accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
        except:
            pass
    try:
        accommodation = accommodation.rstrip(accommodation[-1])
    except:pass
    accommodation_list.append(accommodation)
df['accommodation'] = accommodation_list

In [850]:
df[['accommodation', 'sleepnights']].head(5)

,accommodation,sleepnights
3,"1,3",2
5,"2,3,4",0
6,11,0
9,11,2
10,1,1


In [851]:
## 타겟 컬럼 list (미사용포함)
target_rawcol = []
for i in range(1,22):
    target_rawcol.append("A3_"+str(i))
target_rawcol

['A3_1',
 'A3_2',
 'A3_3',
 'A3_4',
 'A3_5',
 'A3_6',
 'A3_7',
 'A3_8',
 'A3_9',
 'A3_10',
 'A3_11',
 'A3_12',
 'A3_13',
 'A3_14',
 'A3_15',
 'A3_16',
 'A3_17',
 'A3_18',
 'A3_19',
 'A3_20',
 'A3_21']

In [852]:
# Target값 전처리

# Q5. 다녀오신 여행에서 참여하신 활동은 무엇입니까? 해당되는 항목은 모두 선택해 주십시오.
# column명: A5_1 ~ A5_21
# 전처리 전:
# 자연 및 풍경감상
# 음식관광(지역 맛집 등)
# 야외 위락 및 스포츠, 레포츠 활동
# 역사 유적지 방문
# 테마파크, 놀이시설, 동/식물원 방문
# 휴식/휴양
# 온천/스파
# 쇼핑
# 지역 문화예술/공연/전시시설 관람
# 스포츠 경기관람
# 지역 축제/이벤트참가
# 교육/체험 프로그램 참가
# 종교/성지순례
# 카지노, 경마, 경륜 등
# 시티투터
# 드라마 촬영지 방문
# 유흥/오락
# 가족/친지/친구 방문
# 회의참가/시찰
# 교육/훈련/연수
# 기타

# 전처리 후:
# 1.자연 및 풍경감상
# 2.음식관광(지역 맛집 등)
# 3.야외 위락 및 스포츠, 레포츠 활동
# 4.역사 유적지 방문                                  + 16. 드라마 촬영지 방문
# 5.테마파크, 놀이시설, 동/식물원 방문
# 6.휴식/휴양                                            + 7. 온천/스파
# 8.쇼핑                                                   + 15. 시티투어
# 9.지역 문화예술/공연/전시시설 관람         + 10.스포츠 경기관람, 11.지역 축제/이벤트참가 +12. 교육/체험 프로그램 참가
# 17.유흥/오락                                           +14.카지노, 경마, 경륜 등
# 21.기타

# 삭제할 컬럼
# 13.종교/성지순례
# 18.가족/친지/친구 방문
# 19.회의참가/시찰
# 20.교육/훈련/연수

In [853]:
# 로우 드랍 할 target
# 13.종교/성지순례 385
# 18.가족/친지/친구 방문 2755
# 19.회의참가/시찰 244
# 20.교육/훈련/연수 185
# 21.기타 89 합:4081
dropcol = ['A3_13','A3_18','A3_19','A3_20', 'A3_21']
intdropcol = [13,18,19,20,21]
## 안쓰는 target row drop (기준 밖 로우들 제거)
for i, col in enumerate(dropcol):
    df = df[df[col] != intdropcol[i]]
## 안쓰는 target column drop
df.drop(columns = dropcol, inplace=True)
df

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day,area,no_of_place,month,cost,sleepnights,accommodation
5,11010560321_500005,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,0,None,None,None,34360,1,5,57200,0,"2,3,4"
6,11010560321_500010,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,0,None,None,None,31350,1,5,90000,0,11
10,11010600061_900004,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,1,None,None,None,36360,1,9,100000,1,1
14,11010600061_900043,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,0,None,None,None,31370,1,9,50000,0,11
18,11010600071_200025,1,2,2,NaN,NaN,NaN,2.0,2018.0,2.0,...,1,None,None,None,32340,1,2,198000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24063,39020510131_322555,1,2,2,NaN,NaN,NaN,2.0,2018.0,3.0,...,2,None,None,None,21090,1,3,272000,2,"1,3"
24092,3902057079A_1123383,1,2,2,NaN,NaN,NaN,2.0,2018.0,11.0,...,2,None,None,None,36010,1,11,150000,2,"1,3"
24114,3902057161A_748110,1,2,2,NaN,NaN,NaN,2.0,2018.0,8.0,...,1,None,None,None,39010,1,8,50000,1,11
24126,39020600441_1223043,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,0,None,None,None,39010,1,12,30000,0,11


In [854]:
target_col = ['A3_1',
 'A3_2',
 'A3_3',
 'A3_4',
 'A3_5',
 'A3_6',
 'A3_7',
 'A3_8',
 'A3_9',
 'A3_10',
 'A3_11',
 'A3_12',
 'A3_14',
 'A3_15',
 'A3_16',
 'A3_17']
target_intlist = [1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17]
print(len(target_intlist), len(target_col))

16 16


In [855]:
df[target_col]

,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,A3_9,A3_10,A3_11,A3_12,A3_14,A3_15,A3_16,A3_17
5,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,2.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
18,1.0,2.0,3.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24063,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN
24092,1.0,2.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24114,1.0,NaN,3.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24126,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [856]:
##타겟 컬럼 압축 과정

##각 타겟 컬럼들안에 값이 있다면 추출, list에 저장
##새 컬럼에 list 삽입

target_list = []
for index in df.index:
    target_beforesplit=''
    for columns in target_col:
        try: 
            target_beforesplit = target_beforesplit + str(int(df.loc[index, columns])) + ','
        except:
            pass
    target_beforesplit = target_beforesplit.rstrip(target_beforesplit[-1])
    target_list.append(target_beforesplit)
target_list
df['target_beforesplit'] = target_list

In [857]:
## 기존 타겟 컬럼들 drop

## 타겟 컬럼들의 데이터가 압축되어 새 컬럼에 있으니
df.drop(columns=target_col, inplace=True)

In [858]:
## 교통수단 컬럼들 압축

transportaion_list = []
for index in df.index:
    transportation=''
    for columns in ['A5_1', 'A5_2', 'A5_3']:
        try: 
            transportation = transportation + str(int(df.loc[index, columns])) + ','
        except:
            pass
    transportation = transportation.rstrip(transportation[-1])
    transportaion_list.append(transportation)
transportaion_list
df['transportation'] = transportaion_list

In [859]:
## 교통 수단 압축 확인
df[['A5_1', 'A5_2', 'A5_3','transportation']].head(10)

,A5_1,A5_2,A5_3,transportation
5,7.0,8.0,NaN,"7,8"
6,1.0,NaN,NaN,1
10,1.0,NaN,NaN,1
14,6.0,NaN,NaN,6
18,1.0,NaN,NaN,1
19,1.0,NaN,NaN,1
23,1.0,NaN,NaN,1
24,2.0,5.0,NaN,"2,5"
32,1.0,NaN,NaN,1
36,2.0,10.0,1.0,"2,10,1"


In [860]:
#교통수단 컬럼들 제거
df.drop(columns=['A5_1', 'A5_2', 'A5_3'], inplace=True)

In [861]:
## 여행지 선택 이유 컬럼 압축
reason_list = []
for index in df.index:
    reason=''
    for columns in ['A2_1', 'A2_2', 'A2_3']:
        try: 
            reason = reason + str(int(df.loc[index, columns])) + ','
        except:
            pass
    reason = reason.rstrip(reason[-1])
    reason_list.append(reason)
transportaion_list
df['reason'] = reason_list

In [862]:
df.drop(columns=['A2_1', 'A2_2', 'A2_3'], inplace=True)

In [863]:
## 추가 미사용 컬럼 제거
delete_col = ['A7B_1',
 'A7B_2',
 'A7B_3',
 'A7B_4',
 'A7B_5',
 'A7B_6',
 'A7B_7']

In [864]:
## 여행자간 관계 컬럼 압축

relationship_list = []
for index in df.index:
    relationship=''
    for columns in delete_col:
        try: 
            relationship = relationship + str(int(df.loc[index, columns])) + ','
        except:
            pass
    try:
        relationship = relationship.rstrip(relationship[-1])
    except:pass
    relationship_list.append(relationship)
relationship_list
df['relationship_beforesplit'] = relationship_list

In [865]:
df[['A7B_1',
 'A7B_2',
 'A7B_3',
 'A7B_4',
 'A7B_5',
 'A7B_6',
 'A7B_7', 'relationship_beforesplit']].head(20)

,A7B_1,A7B_2,A7B_3,A7B_4,A7B_5,A7B_6,A7B_7,relationship_beforesplit
5,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5
6,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
10,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
14,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
18,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
19,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
23,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
24,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3
32,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
36,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3


In [866]:
## 관계 컬럼 제거
df.drop(columns=delete_col, inplace=True)
df

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,area,no_of_place,month,cost,sleepnights,accommodation,target_beforesplit,transportation,reason,relationship_beforesplit
5,11010560321_500005,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,34360,1,5,57200,0,"2,3,4","1,2,6","7,8","2,14",5
6,11010560321_500010,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,31350,1,5,90000,0,11,"1,2,7",1,"12,4,14",3
10,11010600061_900004,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,36360,1,9,100000,1,1,"1,2,6",1,6,3
14,11010600061_900043,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,31370,1,9,50000,0,11,"2,6,16",6,"4,2,8",3
18,11010600071_200025,1,2,2,NaN,NaN,NaN,2.0,2018.0,2.0,...,32340,1,2,198000,1,1,"1,2,3,6",1,"4,5,10",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24063,39020510131_322555,1,2,2,NaN,NaN,NaN,2.0,2018.0,3.0,...,21090,1,3,272000,2,"1,3","2,8,15","3,5","8,14,5",1
24092,3902057079A_1123383,1,2,2,NaN,NaN,NaN,2.0,2018.0,11.0,...,36010,1,11,150000,2,"1,3","1,2,8","2,3","8,4,14",1
24114,3902057161A_748110,1,2,2,NaN,NaN,NaN,2.0,2018.0,8.0,...,39010,1,8,50000,1,11,"1,3,6",1,"4,5,13",1
24126,39020600441_1223043,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,39010,1,12,30000,0,11,"2,8",6,"5,14,4",3


In [867]:
## 최종으로 남은 컬럼들 코드에서 명칭으로 컬러명 변경

df.rename(columns = {'D_TRA1_SYEAR' : 'year',
                     'A7' : 'number_of_ppl',
                     'A7A' : 'number_of_children',
                     'N_A10' : 'cost_total_trip',
#                      '': 'cost_total_per_person', 인당비용 컬럼은 제거
                     'A12' : 'satisfied',

                     'SEX': 'sex', 
                     'AGE_1YEAR_UNIT': 'age', 
                     'BMON': 'month', 
                     'BSIDO': 'living_area',
                     'area': 'target_area'
                    
                     
                    }, inplace = True)

In [868]:
df.drop(columns='cost', inplace=True)

In [869]:
df = df.reset_index()
df

,index,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,year,...,D_TRA4_S_Day,target_area,no_of_place,month,sleepnights,accommodation,target_beforesplit,transportation,reason,relationship_beforesplit
0,5,11010560321_500005,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,34360,1,5,0,"2,3,4","1,2,6","7,8","2,14",5
1,6,11010560321_500010,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,31350,1,5,0,11,"1,2,7",1,"12,4,14",3
2,10,11010600061_900004,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,36360,1,9,1,1,"1,2,6",1,6,3
3,14,11010600061_900043,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,31370,1,9,0,11,"2,6,16",6,"4,2,8",3
4,18,11010600071_200025,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,32340,1,2,1,1,"1,2,3,6",1,"4,5,10",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,24063,39020510131_322555,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,21090,1,3,2,"1,3","2,8,15","3,5","8,14,5",1
9205,24092,3902057079A_1123383,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,36010,1,11,2,"1,3","1,2,8","2,3","8,4,14",1
9206,24114,3902057161A_748110,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,None,39010,1,8,1,11,"1,3,6",1,"4,5,13",1
9207,24126,39020600441_1223043,1,2,2,1.0,2.0,1.0,2.0,2018.0,...,None,39010,1,12,0,11,"2,8",6,"5,14,4",3


In [870]:
del df['index']

## 카테고리 전처리

In [872]:
# df.to_csv("2018_전처리중(임시).csv")

In [873]:
# df2=pd.read_csv('2018_전처리중(임시).csv')

In [874]:
## target값 split하여 One-hot encoding
for index in df.index:
  target_list_index = df.loc[index,'target_beforesplit'].split(',')
  for targets in target_list_index:
    for i in [1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17]:
      t_col = 'target_'+str(i)
      if targets == str(i):
        df.loc[index,t_col]= 1

In [875]:
df[['target_beforesplit', 'target_1', 'target_2', 'target_3',
       'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9',
       'target_10', 'target_11', 'target_12', 'target_14', 'target_15',
       'target_16', 'target_17']]

,target_beforesplit,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_14,target_15,target_16,target_17
0,"1,2,6",1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,2,7",1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"1,2,6",1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"2,6,16",NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,"1,2,3,6",1.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,"2,8,15",NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
9205,"1,2,8",1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9206,"1,3,6",1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207,"2,8",NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [876]:
## relationship값 split하여 One-hot encoding
for index in df.index:
  target_list_index = df.loc[index,'relationship_beforesplit'].split(',')
  for i, targets in enumerate(target_list_index):
      t_col = 'relationship'+str(i+1)
      df.loc[index,t_col]= targets

In [877]:
## 여행지 장소 split
for index in df.index:
  target_list_index = df.loc[index,'target_area'].split(',')
  for i, targets in enumerate(target_list_index):
      t_col = 'target_area'+str(i+1)
      df.loc[index,t_col]= targets

In [878]:
## 14박 이상 잔 row제거
for i in df.index: 
    if sum(list(map(int, df.loc[index, 'sleepnights'].split(',')))) > 14:
        df.drop(index=i, inplace=True)

In [879]:
## 잔 횟수와(박)과 숙소 일치시키기

## 반복된 숙소 중복제거후 sleepnight당 나올 수 있는 
## accommodation 경우의수 마다 조건으로 처리
## 마지막 예외 제거 : 4row

## 반복된 숫자 줄이는 메서드
def shrink_accom(accom): 
    i = 0
    result = accom
    while i < len(result)-1:
        if result[i] == result[i+1]:
            del result[i]
        else:
            i = i+1
    return result

In [880]:
df.loc[:50,'accommodation']

0     2,3,4
1        11
2         1
3        11
4         1
5         1
6         6
7        11
8         1
9     1,3,4
10       11
11        3
12      2,3
13        1
14      1,3
15       11
16        1
17      1,3
18       11
19        1
20        1
21       11
22       11
23      5,6
24        5
25       11
26      1,3
27        1
28      1,6
29       11
30      1,3
31       11
32        1
33       11
34       11
35       11
36       11
37       11
38       11
39    1,2,3
40        1
41        1
42       11
43        1
44      1,3
45        1
46        1
47        1
48        6
49       11
50       11
Name: accommodation, dtype: object

In [881]:
test = []
for i in range(1,15):
    test.append('A6_'+str(i))
test

['A6_1',
 'A6_2',
 'A6_3',
 'A6_4',
 'A6_5',
 'A6_6',
 'A6_7',
 'A6_8',
 'A6_9',
 'A6_10',
 'A6_11',
 'A6_12',
 'A6_13',
 'A6_14']

In [882]:
df[test]

,A6_1,A6_2,A6_3,A6_4,A6_5,A6_6,A6_7,A6_8,A6_9,A6_10,A6_11,A6_12,A6_13,A6_14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9205,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9206,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [883]:
df = df[df['accommodation'].notna()]

In [884]:
df[['sleepnights','accommodation']].head(60)

,sleepnights,accommodation
0,0,"2,3,4"
1,0,11
2,1,1
3,0,11
4,1,1
5,1,1
6,0,6
7,0,11
8,2,1
9,3,"1,3,4"


In [885]:
df = df[df['accommodation']!='']
df

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,year,D_TRA1_SMONTH,...,target_12,target_10,target_14,relationship1,relationship2,relationship3,target_area1,target_area2,target_area3,target_area4
0,11010560321_500005,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,NaN,NaN,NaN,5,NaN,NaN,34360,NaN,NaN,NaN
1,11010560321_500010,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,NaN,NaN,NaN,3,NaN,NaN,31350,NaN,NaN,NaN
2,11010600061_900004,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,NaN,NaN,NaN,3,NaN,NaN,36360,NaN,NaN,NaN
3,11010600061_900043,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,NaN,NaN,NaN,3,NaN,NaN,31370,NaN,NaN,NaN
4,11010600071_200025,1,2,2,NaN,NaN,NaN,2.0,2018.0,2.0,...,NaN,NaN,NaN,3,NaN,NaN,32340,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9204,39020510131_322555,1,2,2,NaN,NaN,NaN,2.0,2018.0,3.0,...,NaN,NaN,NaN,1,NaN,NaN,21090,NaN,NaN,NaN
9205,3902057079A_1123383,1,2,2,NaN,NaN,NaN,2.0,2018.0,11.0,...,NaN,NaN,NaN,1,NaN,NaN,36010,NaN,NaN,NaN
9206,3902057161A_748110,1,2,2,NaN,NaN,NaN,2.0,2018.0,8.0,...,NaN,NaN,NaN,1,NaN,NaN,39010,NaN,NaN,NaN
9207,39020600441_1223043,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,3,NaN,NaN,39010,NaN,NaN,NaN


In [886]:
df = df.astype('object')

In [887]:
df.dtypes[100:150]

D_TRA4_SDAY        object
D_TRA4_EYEAR       object
D_TRA4_EMONTH      object
D_TRA4_EDAY        object
D_TRA4_1_CITY1     object
D_TRA4_1_TOWN1     object
D_TRA4_1_CITY2     object
D_TRA4_1_TOWN2     object
D_TRA4_1_CITY3     object
D_TRA4_1_TOWN3     object
D_TRA4_1_SYEAR     object
D_TRA4_1_SMONTH    object
D_TRA4_1_SDAY      object
D_TRA4_1_EYEAR     object
D_TRA4_1_EMONTH    object
D_TRA4_1_EDAY      object
D_TRA4_2_CITY1     object
D_TRA4_2_TOWN1     object
D_TRA4_2_SYEAR     object
D_TRA4_2_SMONTH    object
D_TRA4_2_SDAY      object
D_TRA4_2_EYEAR     object
D_TRA4_2_EMONTH    object
D_TRA4_2_EDAY      object
D_TRA4_VAC         object
D_TRA4_COST        object
D_TRA4__CHEAK      object
C1_1               object
C1_2               object
C1_3               object
C1_4               object
C1_5               object
C1_6               object
C1_7               object
C1_8               object
C1_9               object
C1_10              object
C1_TOTAL           object
A1          

In [888]:
df[df['accommodation']==None]

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,year,D_TRA1_SMONTH,...,target_12,target_10,target_14,relationship1,relationship2,relationship3,target_area1,target_area2,target_area3,target_area4


In [890]:
# df.to_csv('2018_전처리중(임시2).csv')

In [891]:
df=pd.read_csv('2018_전처리중(임시2).csv')

In [892]:
df

,Unnamed: 0,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,year,...,target_12,target_10,target_14,relationship1,relationship2,relationship3,target_area1,target_area2,target_area3,target_area4
0,0,11010560321_500005,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,5.0,NaN,NaN,34360,NaN,NaN,NaN
1,1,11010560321_500010,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,3.0,NaN,NaN,31350,NaN,NaN,NaN
2,2,11010600061_900004,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,3.0,NaN,NaN,36360,NaN,NaN,NaN
3,3,11010600061_900043,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,3.0,NaN,NaN,31370,NaN,NaN,NaN
4,4,11010600071_200025,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,3.0,NaN,NaN,32340,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8929,9204,39020510131_322555,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,1.0,NaN,NaN,21090,NaN,NaN,NaN
8930,9205,3902057079A_1123383,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,1.0,NaN,NaN,36010,NaN,NaN,NaN
8931,9206,3902057161A_748110,1,2,2,NaN,NaN,NaN,2.0,2018.0,...,NaN,NaN,NaN,1.0,NaN,NaN,39010,NaN,NaN,NaN
8932,9207,39020600441_1223043,1,2,2,1.0,2.0,1.0,2.0,2018.0,...,NaN,NaN,NaN,3.0,NaN,NaN,39010,NaN,NaN,NaN


In [893]:
## 잔 횟수와(박)과 숙소 일치시키기

## 반복된 숙소 중복제거후 sleepnight당 나올 수 있는 
## accommodation 경우의수 마다 조건으로 처리
## 마지막 예외 제거 : 4row

## 반복된 숫자 줄이는 메서드
def shrink_accom(accom): 
    i = 0
    result = accom
    while i < len(result)-1:
        if result[i] == result[i+1]:
            del result[i]
        else:
            i = i+1
    return result

In [894]:
for index in df.index: ##당일여행 한번 갔을때: accommodation=0
    accommo_list = []
    try:splitted_nights = list(map(int, df.loc[index, 'sleepnights'].split(','))) ## sleepnights를 int로 split
    except:print("typeerror>>",index)
    if splitted_nights == [0]:
        df.loc[index, 'accommodation'] = 0
    else:
        s = np.count_nonzero(splitted_nights) ## splitted_nights중 0이 아닌 수 count s
        if s == 0: ## 0,0,0 와 같이 0만 있을 때, 0으로 채움            for i in range(len(splitted_nights)):
            for i in range(len(splitted_nights)):
                accommo_list.append(0)
            for i, accommo in enumerate(accommo_list):
                col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                df.loc[index,col_name] = 0
        else:
            try:splitted_accommo = list(map(int, df.loc[index, 'accommodation'].split(',')))
            except:print('FLOAT>>',index, df.loc[index,'accommodation'])
            try:shrinked_accommo = shrink_accom(list(map(int, df.loc[index, 'accommodation'].split(',')))) ## 반복된 숫자 줄여줌
            except:print('FLOAT>>',index, df.loc[index,'accommodation'])
            if (s == len(shrinked_accommo)) or (len(shrinked_accommo) == 1):## s와 accommo의 길이가 같을때: s의 수대로 accommo를 곱해준다
                if [len(shrinked_accommo)] == splitted_nights: ## 0미포함
                    for i, accommo in enumerate(shrinked_accommo):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                else:    ##0포함
                    j=0
                    for i, nights in enumerate(splitted_nights):
                        if nights == 0:
                            accommo_list.append(0)
                        else:
                            for ni in range(nights):
                                accommo_list.append(shrinked_accommo[j])
                    j+=1
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
            elif s > len(shrinked_accommo): ## s가 accommo의 길이보다 클 때
                if len(list(map(int, df.loc[index, 'accommodation'].split(',')))) == len(splitted_nights):
                    for i, accommo in enumerate(splitted_nights):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                    
                else:
                    for i, nights in enumerate(splitted_nights): ## 3,0,1 12,0,1 => 12,12,12,0,1
                        if nights == 0:
                            accommo_list.append(0)
                        else:
                            for ni in range(nights):
                                try:accommo_list.append(shrinked_accommo[i])
                                except:print('indexError>>', index, "s:", s, "s_night:", splitted_nights , 
                                             "slpnight:", df.loc[index, 'sleepnights'], 
                                             "accom", df.loc[index, 'accommodation'], 
                                             "splt_accom:", splitted_accommo)
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo 
                
            else: 
                if sum(list(map(int, df.loc[1215, 'sleepnights'].split(',')))) == len(list(map(
                    int, df.loc[index, 'accommodation'].split(',')))): ## [0,0,2] [1,2]
                    j=0
                    for i, nights in enumerate(list(map(int, df.loc[1215, 'sleepnights'].split(',')))): 
                        if nights == 0:
                            accommo_list.append(0)
                        else:
                            for ni in range(nights):
                                accommo_list.append(list(map(int, df.loc[index, 'accommodation'].split(',')))[j])
                                j+=1
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo 
                if [len(shrinked_accommo)] == splitted_nights: ## 0미포함
                    for i, accommo in enumerate(shrinked_accommo):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                elif sum(list(map(int, df.loc[index, 'sleepnights'].split(',')))) == len(list(map(int, df.loc[index, 'accommodation'].split(',')))):
                    accommo_list = list(map(int, df.loc[index, 'accommodation'].split(',')))
                    for i, accommo in enumerate(accommo_list):
                        col_name = 'accommodation_' + str(i+1) ##dataframe에 입력
                        df.loc[index,col_name] = accommo
                    # print("caught row>>", index, "s:", s, "s_night:", splitted_nights , "slpnight:", df.loc[index, 'sleepnights'], "accom", df.loc[index, 'accommodation'], "shrink_accom:", shrinked_accommo)
                else:
                    print("drop row>>", index, "s:", s, "s_night:", splitted_nights , "slpnight:", df.loc[index, 'sleepnights'], "accom", df.loc[index, 'accommodation'], "shrink_accom:", shrinked_accommo)
                    df.drop(index=index, inplace=True)
                


drop row>> 17 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 23 s: 1 s_night: [4] slpnight: 4 accom 5,6 shrink_accom: [5, 6]
drop row>> 26 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 28 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 30 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 39 s: 1 s_night: [2] slpnight: 2 accom 1,2,3 shrink_accom: [1, 2, 3]
drop row>> 44 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 54 s: 1 s_night: [4] slpnight: 4 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 63 s: 1 s_night: [1] slpnight: 1 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 74 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 82 s: 1 s_night: [3] slpnight: 3 accom 1,3 shrink_accom: [1, 3]
drop row>> 85 s: 1 s_night: [3] slpnight: 3 accom 1,3 shrink_accom: [1, 3]
drop row>> 88 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop 

drop row>> 1092 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 1094 s: 1 s_night: [1] slpnight: 1 accom 1,5 shrink_accom: [1, 5]
drop row>> 1111 s: 1 s_night: [1] slpnight: 1 accom 1,5,6 shrink_accom: [1, 5, 6]
drop row>> 1114 s: 1 s_night: [3] slpnight: 3 accom 1,2,3,4 shrink_accom: [1, 2, 3, 4]
drop row>> 1118 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 1125 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 1131 s: 1 s_night: [1] slpnight: 1 accom 1,2 shrink_accom: [1, 2]
drop row>> 1160 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 1163 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 1166 s: 1 s_night: [1] slpnight: 1 accom 1,7 shrink_accom: [1, 7]
drop row>> 1173 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 1179 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 1192 s: 1 s_night: [1] slpnigh

drop row>> 2379 s: 1 s_night: [3] slpnight: 3 accom 1,3,4,7 shrink_accom: [1, 3, 4, 7]
drop row>> 2392 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 2399 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 2406 s: 1 s_night: [2] slpnight: 2 accom 1,6,7 shrink_accom: [1, 6, 7]
drop row>> 2432 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 2463 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 2466 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 2479 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 2507 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 2514 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 2527 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 2535 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 2541 s: 1 s_night: [1] sl

drop row>> 3587 s: 1 s_night: [1] slpnight: 1 accom 1,7 shrink_accom: [1, 7]
drop row>> 3592 s: 1 s_night: [2] slpnight: 2 accom 1,3,4,5 shrink_accom: [1, 3, 4, 5]
drop row>> 3604 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 3620 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 3623 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 3629 s: 1 s_night: [4] slpnight: 4 accom 1,3 shrink_accom: [1, 3]
drop row>> 3658 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 3663 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 3709 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 3738 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 3756 s: 1 s_night: [1] slpnight: 1 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 3764 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 3769 s: 1 s_night: [1] slpnight: 1 accom 1,3 

drop row>> 4772 s: 1 s_night: [1] slpnight: 1 accom 1,7 shrink_accom: [1, 7]
drop row>> 4781 s: 1 s_night: [5] slpnight: 5 accom 1,6 shrink_accom: [1, 6]
drop row>> 4784 s: 1 s_night: [1] slpnight: 1 accom 1,3,6 shrink_accom: [1, 3, 6]
drop row>> 4800 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 4824 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 4838 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 4850 s: 1 s_night: [1] slpnight: 1 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 4854 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 4859 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 4861 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 4863 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 4865 s: 1 s_night: [1] slpnight: 1 accom 1,7 shrink_accom: [1, 7]
drop row>> 4867 s: 1 s_night: [1] slpnight: 1 accom

drop row>> 5758 s: 1 s_night: [4] slpnight: 4 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 5762 s: 1 s_night: [2] slpnight: 2 accom 1,3,4,5 shrink_accom: [1, 3, 4, 5]
drop row>> 5763 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 5767 s: 1 s_night: [1] slpnight: 1 accom 1,4 shrink_accom: [1, 4]
drop row>> 5770 s: 1 s_night: [1] slpnight: 1 accom 1,4 shrink_accom: [1, 4]
drop row>> 5799 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 5800 s: 1 s_night: [1] slpnight: 1 accom 1,7 shrink_accom: [1, 7]
drop row>> 5806 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 5807 s: 1 s_night: [1] slpnight: 1 accom 1,4 shrink_accom: [1, 4]
drop row>> 5813 s: 1 s_night: [1] slpnight: 1 accom 1,2,4,5 shrink_accom: [1, 2, 4, 5]
drop row>> 5815 s: 1 s_night: [1] slpnight: 1 accom 1,2,3,5 shrink_accom: [1, 2, 3, 5]
drop row>> 5834 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 5856 s: 1 s_nig

drop row>> 7165 s: 1 s_night: [1] slpnight: 1 accom 1,6 shrink_accom: [1, 6]
drop row>> 7168 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 7181 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7184 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7199 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7201 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7213 s: 1 s_night: [1] slpnight: 1 accom 1,4 shrink_accom: [1, 4]
drop row>> 7275 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7293 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7298 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7320 s: 1 s_night: [1] slpnight: 1 accom 1,2 shrink_accom: [1, 2]
drop row>> 7329 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 7339 s: 1 s_night: [2] sl

drop row>> 8461 s: 1 s_night: [3] slpnight: 3 accom 1,2,3,4 shrink_accom: [1, 2, 3, 4]
drop row>> 8477 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 8480 s: 1 s_night: [2] slpnight: 2 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 8481 s: 1 s_night: [3] slpnight: 3 accom 1,2,3,4,7 shrink_accom: [1, 2, 3, 4, 7]
drop row>> 8483 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 8484 s: 1 s_night: [2] slpnight: 2 accom 1,2,3,4 shrink_accom: [1, 2, 3, 4]
drop row>> 8502 s: 1 s_night: [2] slpnight: 2 accom 1,2,3,4 shrink_accom: [1, 2, 3, 4]
drop row>> 8506 s: 1 s_night: [2] slpnight: 2 accom 1,6,7 shrink_accom: [1, 6, 7]
drop row>> 8527 s: 1 s_night: [1] slpnight: 1 accom 1,3,4 shrink_accom: [1, 3, 4]
drop row>> 8539 s: 1 s_night: [1] slpnight: 1 accom 1,3 shrink_accom: [1, 3]
drop row>> 8548 s: 1 s_night: [1] slpnight: 1 accom 1,4 shrink_accom: [1, 4]
drop row>> 8556 s: 1 s_night: [1] slpnight: 1 accom 1,2 shrink_accom: [1, 2]
drop row>> 

In [895]:
for index in df.index:
  target_list_index = df.loc[index,'transportation'].split(',')
  for i, targets in enumerate(target_list_index):
       t_col = 'transportation'+str(i+1)
       df.loc[index,t_col]= targets

In [896]:
for index in df.index:
  target_list_index = df.loc[index,'reason'].split(',')
  for i, targets in enumerate(target_list_index):
       t_col = 'reason'+str(i+1)
       df.loc[index,t_col]= targets

In [897]:
df['cost_total_per_person']=None
# 인당 지출비용 임시컬럼 추가

In [898]:
## 컬럼 순서 정리

column_order = ['year', 'target_area', 'no_of_place', 'sleepnights', 'accommodation',
       'number_of_ppl', 'number_of_children', 'cost_total_trip',
        'cost_total_per_person',
       'satisfied', 'sex', 'age', 'month',
       'living_area', 'target_beforesplit', 'transportation', 'reason',
       'relationship_beforesplit','relationship1', 'relationship2',
       'relationship3', 'target_1', 'target_2', 'target_3','target_4','target_5','target_6', 'target_7','target_8', 'target_9','target_10','target_11', 'target_12','target_14', 'target_15','target_16', 'target_17', 
        'target_area1', 'target_area2', 'target_area3',
       'target_area4', 'target_area5', 'target_area6', 'target_area7',
       'target_area8', 'target_area9', 'target_area10', 'target_area11',
       'transportation1', 'transportation2', 'transportation3', 'reason1',
       'reason2', 'reason3', 'accommodation_1', 'accommodation_2',
       'accommodation_3', 'accommodation_4', 'accommodation_5',
       'accommodation_6', 'accommodation_7', 'accommodation_8',
       'accommodation_9', 'accommodation_10', 'accommodation_11',
       'accommodation_12', 'accommodation_13', 'accommodation_14']

In [899]:
df = df.reindex(columns=column_order)

In [900]:
df

,year,target_area,no_of_place,sleepnights,accommodation,number_of_ppl,number_of_children,cost_total_trip,cost_total_per_person,satisfied,...,accommodation_5,accommodation_6,accommodation_7,accommodation_8,accommodation_9,accommodation_10,accommodation_11,accommodation_12,accommodation_13,accommodation_14
0,2018.0,34360,1,0,0,33.0,0.0,57200.000000,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018.0,31350,1,0,0,5.0,0.0,90000.000000,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018.0,36360,1,1,1,5.0,0.0,42003.514938,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018.0,31370,1,0,0,2.0,0.0,50000.000000,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018.0,32340,1,1,1,5.0,0.0,198000.000000,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8929,2018.0,21090,1,2,"1,3",2.0,0.0,263111.111111,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8930,2018.0,36010,1,2,"1,3",3.0,1.0,140945.674044,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8931,2018.0,39010,1,1,11,3.0,1.0,45283.018868,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8932,2018.0,39010,1,0,0,5.0,0.0,30000.000000,None,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [901]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8143 entries, 0 to 8933
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      8128 non-null   float64
 1   target_area               8143 non-null   object 
 2   no_of_place               8143 non-null   int64  
 3   sleepnights               8143 non-null   object 
 4   accommodation             8143 non-null   object 
 5   number_of_ppl             8143 non-null   float64
 6   number_of_children        8003 non-null   float64
 7   cost_total_trip           8143 non-null   float64
 8   cost_total_per_person     0 non-null      object 
 9   satisfied                 8143 non-null   float64
 10  sex                       8143 non-null   int64  
 11  age                       8143 non-null   int64  
 12  month                     8143 non-null   int64  
 13  living_area               8143 non-null   int64  
 14  target_b

In [903]:
# df.to_csv('2018_전처리 최종.csv')

In [904]:
df=pd.read_csv('2018_전처리 최종.csv')

In [905]:
df

,Unnamed: 0,year,target_area,no_of_place,sleepnights,accommodation,number_of_ppl,number_of_children,cost_total_trip,cost_total_per_person,...,accommodation_5,accommodation_6,accommodation_7,accommodation_8,accommodation_9,accommodation_10,accommodation_11,accommodation_12,accommodation_13,accommodation_14
0,0,2018.0,34360,1,0,0,33.0,0.0,57200.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018.0,31350,1,0,0,5.0,0.0,90000.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018.0,36360,1,1,1,5.0,0.0,42003.514938,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018.0,31370,1,0,0,2.0,0.0,50000.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018.0,32340,1,1,1,5.0,0.0,198000.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8138,8929,2018.0,21090,1,2,"1,3",2.0,0.0,263111.111111,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8139,8930,2018.0,36010,1,2,"1,3",3.0,1.0,140945.674044,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8140,8931,2018.0,39010,1,1,11,3.0,1.0,45283.018868,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8141,8932,2018.0,39010,1,0,0,5.0,0.0,30000.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [908]:
del df['Unnamed: 0']

In [909]:
df

,year,target_area,no_of_place,sleepnights,accommodation,number_of_ppl,number_of_children,cost_total_trip,cost_total_per_person,satisfied,...,accommodation_5,accommodation_6,accommodation_7,accommodation_8,accommodation_9,accommodation_10,accommodation_11,accommodation_12,accommodation_13,accommodation_14
0,2018.0,34360,1,0,0,33.0,0.0,57200.000000,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018.0,31350,1,0,0,5.0,0.0,90000.000000,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018.0,36360,1,1,1,5.0,0.0,42003.514938,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018.0,31370,1,0,0,2.0,0.0,50000.000000,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018.0,32340,1,1,1,5.0,0.0,198000.000000,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8138,2018.0,21090,1,2,"1,3",2.0,0.0,263111.111111,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8139,2018.0,36010,1,2,"1,3",3.0,1.0,140945.674044,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8140,2018.0,39010,1,1,11,3.0,1.0,45283.018868,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8141,2018.0,39010,1,0,0,5.0,0.0,30000.000000,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [910]:
df.columns

Index(['year', 'target_area', 'no_of_place', 'sleepnights', 'accommodation',
       'number_of_ppl', 'number_of_children', 'cost_total_trip',
       'cost_total_per_person', 'satisfied', 'sex', 'age', 'month',
       'living_area', 'target_beforesplit', 'transportation', 'reason',
       'relationship_beforesplit', 'relationship1', 'relationship2',
       'relationship3', 'target_1', 'target_2', 'target_3', 'target_4',
       'target_5', 'target_6', 'target_7', 'target_8', 'target_9', 'target_10',
       'target_11', 'target_12', 'target_14', 'target_15', 'target_16',
       'target_17', 'target_area1', 'target_area2', 'target_area3',
       'target_area4', 'target_area5', 'target_area6', 'target_area7',
       'target_area8', 'target_area9', 'target_area10', 'target_area11',
       'transportation1', 'transportation2', 'transportation3', 'reason1',
       'reason2', 'reason3', 'accommodation_1', 'accommodation_2',
       'accommodation_3', 'accommodation_4', 'accommodation_5',
       